In [11]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

df = pd.read_csv('/Users/varnika/Desktop/harvard/all_var_int.csv')
pd.set_option('display.max_columns', None)


/var/folders/gw/xg9wcgln7x1f2k7qpz6vkm_c0000gn/T/ipykernel_42253/3777261934.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/varnika/Desktop/harvard/all_var_int.csv')


In [17]:
landslide_vars = [
    "rugged", "elev_mean", 
    "bdod", "cec", "cfvo", "clay", "sand", "silt", "phh_20", "soc", "ocd", "ocs",
    "floss_ratio", "forest_gain_ratio", "crp_ratio", "fcover_ratio", 'landslide_f', 'rainfall_2020' 
]



In [20]:
df['floss_ratio'] = df['floss_total'] / df['geog_area']
df['crp_ratio']   = df['crp_sq_km']   / df['geog_area']
df['fcover_ratio'] = df['fcover_sq_km'] / df['geog_area']
df['forest_gain_ratio'] = df['forest_gain_fraction'] / df['geog_area']


In [21]:
# 1. Subset the relevant columns for landslides
landslide_cols = ["state_code", "state_name", "district_code", "district_name",
                  "block_code", "block_name", "stcode11", "dtcode11", "blkcode11",
                  "FID_Key"] + landslide_vars

landslide_df = df[landslide_cols].copy()


# 2. Initialize scaler
Rscaler = RobustScaler()

# 3. Fit + transform only the features, not the IDs
features_only = landslide_df[landslide_vars].values
features_scaled = Rscaler.fit_transform(features_only)

# 4. Create a scaled DataFrame
landslide_scaled = pd.DataFrame(features_scaled, 
                                columns=landslide_vars, 
                                index=landslide_df.index)

# Merge scaled columns back or keep them separate
for col in landslide_vars:
    landslide_df[col + "_scaled"] = landslide_scaled[col]

In [22]:
# Drop ID columns and keep only scaled columns for PCA
ls_pca_vars = [col for col in landslide_df.columns if col.endswith("_scaled")]
X_landslide = landslide_df[ls_pca_vars].astype(float)

pca = PCA()
pca.fit(X_landslide)

# Summarize variance
pcaSummary_ls = pd.DataFrame({
    '% variance': pca.explained_variance_ratio_,
    'Cumulative %': np.cumsum(pca.explained_variance_ratio_)
})

print(pcaSummary_ls.round(3))

# Suppose you keep 2 or 3 PCs based on variance
n_components = 2  # or 3
pca_n = PCA(n_components=n_components)
X_landslide_pca = pca_n.fit_transform(X_landslide)

# Weights of each PC
pca_weights = pcaSummary_ls['% variance'][:n_components]

# Store principal components in DataFrame
for i in range(n_components):
    landslide_df[f"Landslide_PC{i+1}"] = X_landslide_pca[:, i]

# Weighted sum => “Landslide_Exposure_Index”
landslide_df["Landslide_Exposure_Index"] = 0
for i in range(n_components):
    landslide_df["Landslide_Exposure_Index"] += pca_weights.iloc[i] * landslide_df[f"Landslide_PC{i+1}"]

# Inspect final index
print(landslide_df[["block_code", "Landslide_Exposure_Index"]].head())
print(landslide_df["Landslide_Exposure_Index"].describe())

    % variance  Cumulative %
0        0.552         0.552
1        0.245         0.797
2        0.194         0.991
3        0.003         0.994
4        0.001         0.996
5        0.001         0.997
6        0.001         0.998
7        0.001         0.998
8        0.000         0.999
9        0.000         0.999
10       0.000         0.999
11       0.000         1.000
12       0.000         1.000
13       0.000         1.000
14       0.000         1.000
15       0.000         1.000
16       0.000         1.000
17       0.000         1.000
   block_code  Landslide_Exposure_Index
0        6498                 12.869374
1        6492                  7.462357
2        4689                 -4.338322
3        4690                 -4.653807
4        4692                 -4.641996
count    5.815000e+03
mean    -1.564049e-16
std      1.401345e+01
min     -4.729436e+00
25%     -4.611154e+00
50%     -4.331392e+00
75%     -2.584122e+00
max      1.932648e+02
Name: Landslide_Exposure_Index, d

In [24]:
# keeping 3 PCs based on variance
n_components = 3  
pca_n = PCA(n_components=n_components)
X_landslide_pca = pca_n.fit_transform(X_landslide)

pca_weights = pcaSummary_ls['% variance'][:n_components]

# Store principal components in DataFrame
for i in range(n_components):
    landslide_df[f"Landslide_PC{i+1}"] = X_landslide_pca[:, i]


In [25]:
landslide_df["Landslide_Exposure_Index"] = 0
for i in range(n_components):
    landslide_df["Landslide_Exposure_Index"] += (
        pca_weights.iloc[i] * landslide_df[f"Landslide_PC{i+1}"]
    )

In [26]:
print(landslide_df.columns)


Index(['state_code', 'state_name', 'district_code', 'district_name',
       'block_code', 'block_name', 'stcode11', 'dtcode11', 'blkcode11',
       'FID_Key', 'rugged', 'elev_mean', 'bdod', 'cec', 'cfvo', 'clay', 'sand',
       'silt', 'phh_20', 'soc', 'ocd', 'ocs', 'floss_ratio',
       'forest_gain_ratio', 'crp_ratio', 'fcover_ratio', 'landslide_f',
       'rainfall_2020', 'rugged_scaled', 'elev_mean_scaled', 'bdod_scaled',
       'cec_scaled', 'cfvo_scaled', 'clay_scaled', 'sand_scaled',
       'silt_scaled', 'phh_20_scaled', 'soc_scaled', 'ocd_scaled',
       'ocs_scaled', 'floss_ratio_scaled', 'forest_gain_ratio_scaled',
       'crp_ratio_scaled', 'fcover_ratio_scaled', 'landslide_f_scaled',
       'rainfall_2020_scaled', 'Landslide_PC1', 'Landslide_PC2',
       'Landslide_Exposure_Index', 'Landslide_PC3'],
      dtype='object')


In [27]:
loadings = pd.DataFrame(pca_n.components_.T,
   index=X_landslide.columns,
   columns=[f"PC{i+1}" for i in range(n_components)]
)
print(loadings)


                               PC1       PC2       PC3
rugged_scaled             0.295190  0.523193 -0.793068
elev_mean_scaled          0.015378  0.046026 -0.052936
bdod_scaled              -0.014597 -0.009110  0.010229
cec_scaled               -0.004643 -0.002723  0.000012
cfvo_scaled               0.008079  0.007082 -0.011737
clay_scaled              -0.000373 -0.005708  0.006972
sand_scaled               0.001983  0.000895 -0.008221
silt_scaled              -0.001540  0.004754  0.001049
phh_20_scaled            -0.018668 -0.008993  0.005121
soc_scaled                0.013746  0.020362 -0.024455
ocd_scaled                0.011676  0.017951 -0.019929
ocs_scaled                0.015862  0.014101 -0.014313
floss_ratio_scaled        0.943536 -0.286901  0.162846
forest_gain_ratio_scaled  0.142419  0.800083  0.582475
crp_ratio_scaled         -0.012067 -0.010303  0.010729
fcover_ratio_scaled       0.021390  0.014624 -0.014657
landslide_f_scaled        0.007267  0.013114 -0.022370
rainfall_2

In [30]:


# Weighted sum => “Landslide_Exposure_Index” using the 3 PCs
landslide_df["Landslide_Exposure_Index"] = (
    pca_weights.iloc[0] * landslide_df["Landslide_PC1"] +
    pca_weights.iloc[1] * landslide_df["Landslide_PC2"] +
    pca_weights.iloc[2] * landslide_df["Landslide_PC3"]
)

# Inspect a few rows
print(landslide_df[["block_code", "Landslide_Exposure_Index"]].head())

# Summary stats (min, max, mean, etc.)
print(landslide_df["Landslide_Exposure_Index"].describe())


   block_code  Landslide_Exposure_Index
0        6498                 13.477031
1        6492                  8.238749
2        4689                 -4.244717
3        4690                 -4.421473
4        4692                 -4.423521
count    5.815000e+03
mean     5.474173e-16
std      1.429118e+01
min     -4.478636e+00
25%     -4.369198e+00
50%     -4.102623e+00
75%     -2.483507e+00
max      2.747098e+02
Name: Landslide_Exposure_Index, dtype: float64
